In [1]:
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path
from inaSpeechSegmenter import Segmenter
from inaSpeechSegmenter.export_funcs import seg2csv, seg2textgrid
from spectralcluster import SpectralClusterer
from spectralcluster import RefinementOptions
from spectralcluster import ThresholdType
from spectralcluster import ICASSP2018_REFINEMENT_SEQUENCE
from resemblyzer.audio import sampling_rate
from pydub import AudioSegment
import numpy as np
import os
import joblib
import librosa
import pandas as pd

## Définition de la fonction générale

In [2]:
def count_speakers1(audio_path):
    
    def extract_speech(audio_path):
        segmentation = Segmenter(detect_gender=False)
        return segmentation(audio_path)
    
    def concatenate_segments(segmentation):
        t1 = [] #Temps de début de parole
        t2 = [] #Temps de fin de parole

        #Remplissage des tableaux précédents
        for i in range(len(segmentation)):
            #if (segmentation[i][0] == 'male') or (segmentation[i][0] == 'female'):
            if segmentation[i][0] == 'speech':
                t1.append(segmentation[i][1])
                t2.append(segmentation[i][2])

        #Importation du fichier .wav dont on veut les différents locuteurs
        newAudio = AudioSegment.from_wav(audio_path)
        #Génération de deux fichiers audio vides
        audio = AudioSegment.empty()

        # Génération fichier
        for i in range(len(t1)):
            audio += newAudio[t1[i]*1000:t2[i]*1000]
        audio.export(out_f = "cut_audio.wav", 
                            format = "wav")
        return "cut_audio.wav"
    
    def speaker_segmentation(audio):
        labelling = []
        if os.stat(audio).st_size > 100000 :
            wav_fpath = Path(audio)
            wav = preprocess_wav(wav_fpath)
            encoder = VoiceEncoder("cpu")
            _, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)

            # Pour déterminer les différents locuteurs/locutrices, on clusterise
            refinement_options = RefinementOptions(gaussian_blur_sigma=1,
                                                    p_percentile=0.90,
                                                    thresholding_soft_multiplier=0.01,
                                                    thresholding_type=ThresholdType.RowMax,
                                                    refinement_sequence=ICASSP2018_REFINEMENT_SEQUENCE)

            clusterer = SpectralClusterer(min_clusters=2,
                                          max_clusters=100,
                                          refinement_options=refinement_options)

            labels = clusterer.predict(cont_embeds)

            times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
            start_time = 0

            for i,time in enumerate(times):
                if i>0 and labels[i]!=labels[i-1]:
                    temp = [str(labels[i-1]),start_time,time]
                    labelling.append(tuple(temp))
                    start_time = time
                if i==len(times)-1:
                    temp = [str(labels[i]),start_time,time]
                    labelling.append(tuple(temp))
        else:
            labelling.append(-1)
        return labelling
    
    def extract_audios(labelling):
        audio = AudioSegment.from_wav(audio_path)
        L=[]
        for t in labelling:
            if not(t == -1):
                new_audio = AudioSegment.empty()
                new_audio = audio[t[1]*1000:t[2]*1000]
                out_f= "new_audio"+str([t[1],t[2]])+".wav"
                new_audio.export(out_f , format = "wav")
                L.append(out_f)
        return(L)
    
    def different_speakers(audio_path):
    
        ## Exécution partie 1 : 
        # Etape I : 
        segmentation = extract_speech(audio_path)
        # Etape II : 
        newAudio = concatenate_segments(segmentation)
        # Etape III :
        labelling = speaker_segmentation(newAudio)
        # Etape IV : 
        resultat = extract_audios(labelling)
            
        return [resultat, labelling]

    def get_char_son(fichier):
        signal, sr=librosa.load(fichier)
        S = np.abs(librosa.stft(signal))
        comps, acts = librosa.decompose.decompose(S, n_components=1)
        data=pd.DataFrame([np.ravel(comps).tolist()])
        data['ID']=fichier
        return(data.set_index('ID'))
    
    def predict_sound(tab):
        loaded_model = joblib.load('Reconnaissance Femme_Homme\my_model.pkl')
        if loaded_model.predict(tab)[0]==0:
            return('Homme')
        return('Femme')
    
    part1 = different_speakers(audio_path)
    print(part1)
    files = part1[0]
    labelling = part1[1]
    dict_speaker = {}
    dict_speaker_sex = {}
    for i in range(len(files)):
        tab = get_char_son(files[i])
        pred_sex = predict_sound(tab)
        try:
            dict_speaker[labelling[i]] = 1 + dict_speaker[labelling[i]]
        except:
            dict_speaker[labelling[i]] = 1
        dict_speaker_sex[labelling[i]] = pred_sex

    c_male = 0
    c_female = 0 
    for cle,valeur in dict_speaker_sex.items():
        if valeur == 'Homme':
            c_male = c_male + dict_speaker[cle]
        elif valeur == 'Femme':
            c_female = c_female + dict_speaker[cle]

    result = {'M':c_male , 'F':c_female}
    return result
    

In [3]:
count_speakers1("moviesoundclips.net_test/moviesoundclips.net/959.wav")

C:\Users\Paul\anaconda3\lib\site-packages\pyannote\algorithms\utils\viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
C:\Users\Paul\anaconda3\lib\site-packages\pyannote\algorithms\utils\viterbi.py:95: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
C:\Users\Paul\anaconda3\lib\site-packages\inaSpeechSegmenter\segmenter.py:61: RuntimeWarning: invalid value encountered in subtract
  data = (data - np.mean(data, axis=1).reshape((len(data), 1))) / np.std(data, axis=1).reshape((len(data), 1))
C:\Users\Paul\anaconda3\lib\site-packages\numpy\core\_methods.py:229: RuntimeWarning: invalid value encountered in subtract

Loaded the voice encoder model on cpu in 0.08 seconds.


C:\Users\Paul\anaconda3\lib\site-packages\resemblyzer\audio.py:47: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.], sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  frames = librosa.feature.melspectrogram(


[['new_audio[0, 2.0].wav', 'new_audio[2.0, 2.66].wav', 'new_audio[2.66, 4.1].wav', 'new_audio[4.1, 5.42].wav', 'new_audio[5.42, 6.5].wav', 'new_audio[6.5, 7.76].wav', 'new_audio[7.76, 8.78].wav', 'new_audio[8.78, 9.8].wav'], [('4', 0, 2.0), ('6', 2.0, 2.66), ('3', 2.66, 4.1), ('2', 4.1, 5.42), ('7', 5.42, 6.5), ('1', 6.5, 7.76), ('0', 7.76, 8.78), ('5', 8.78, 9.8)]]


C:\Users\Paul\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "
C:\Users\Paul\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"
C:\Users\Paul\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "
C:\Users\Paul\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1090: ConvergenceWarning: Maximum number of iterations

{'M': 8, 'F': 0}

## Liste des fichiers à tester

In [4]:
import os
res={}
L=os.listdir('moviesoundclips.net_test/moviesoundclips.net/')
print(L)

['100.wav', '1065.wav', '1156.wav', '1162.wav', '1167.wav', '1226.wav', '125.wav', '126.wav', '127.wav', '1310.wav', '1313.wav', '1327.wav', '137.wav', '15.wav', '1508.wav', '1532.wav', '1551.wav', '156.wav', '157.wav', '161.wav', '1666.wav', '1727.wav', '173.wav', '1730.wav', '1731.wav', '1754.wav', '1757.wav', '1758.wav', '1770.wav', '1777.wav', '1819.wav', '1826.wav', '1827.wav', '1832.wav', '1875.wav', '1892.wav', '1976.wav', '1990.wav', '1994.wav', '1996.wav', '2062.wav', '2071.wav', '2087.wav', '2097.wav', '2129.wav', '2134.wav', '2159.wav', '2163.wav', '2164.wav', '2191.wav', '2195.wav', '220.wav', '2213.wav', '2296.wav', '2317.wav', '2322.wav', '2375.wav', '2382.wav', '2389.wav', '2392.wav', '2395.wav', '240.wav', '2406.wav', '2428.wav', '2430.wav', '2453.wav', '2455.wav', '2458.wav', '2463.wav', '2470.wav', '2481.wav', '2538.wav', '2542.wav', '2546.wav', '2553.wav', '2569.wav', '2636.wav', '2667.wav', '2693.wav', '2696.wav', '2704.wav', '2724.wav', '2751.wav', '2753.wav', '280

# Test sur le jeu de données

In [ ]:
res3={}
for i in range(len(L)):
    sep=(L[i].find('.'))
    num=L[i][:sep]
    try:
        res_prov=count_speakers1("moviesoundclips.net_test/moviesoundclips.net/"+L[i])
        res3[str(num)]=res_prov
    except:
        print('Erreur')
    print(i/len(L))

C:\Users\Paul\anaconda3\lib\site-packages\pyannote\algorithms\utils\viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
C:\Users\Paul\anaconda3\lib\site-packages\pyannote\algorithms\utils\viterbi.py:95: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
C:\Users\Paul\anaconda3\lib\site-packages\resemblyzer\audio.py:33: FutureWarning: Pass orig_sr=11025, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  wav = librosa.resample(wav, source_sr, sampling_rate)


Loaded the voice encoder model on cpu in 0.01 seconds.


C:\Users\Paul\anaconda3\lib\site-packages\resemblyzer\audio.py:47: FutureWarning: Pass y=[0.02292936 0.02088435 0.01217182 ... 0.         0.         0.        ], sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  frames = librosa.feature.melspectrogram(


[['new_audio[0, 1.46].wav', 'new_audio[1.46, 2.84].wav', 'new_audio[2.84, 3.2].wav', 'new_audio[3.2, 4.34].wav', 'new_audio[4.34, 6.08].wav', 'new_audio[6.08, 6.62].wav', 'new_audio[6.62, 7.94].wav', 'new_audio[7.94, 8.84].wav', 'new_audio[8.84, 10.64].wav'], [('4', 0, 1.46), ('6', 1.46, 2.84), ('7', 2.84, 3.2), ('0', 3.2, 4.34), ('1', 4.34, 6.08), ('7', 6.08, 6.62), ('2', 6.62, 7.94), ('5', 7.94, 8.84), ('3', 8.84, 10.64)]]


C:\Users\Paul\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "
C:\Users\Paul\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "
C:\Users\Paul\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"
C:\Users\Paul\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None'

...

C:\Users\Paul\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "
C:\Users\Paul\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1090: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn("Maximum number of iterations %d reached. Increase it to"
C:\Users\Paul\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:312: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(("The 'init' value, when 'init=None' and "
C:\Users\Paul\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1090: ConvergenceWarning: Maximum number of iterations

Erreur
0.1774193548387097


C:\Users\Paul\anaconda3\lib\site-packages\pyannote\algorithms\utils\viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
C:\Users\Paul\anaconda3\lib\site-packages\pyannote\algorithms\utils\viterbi.py:95: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
C:\Users\Paul\anaconda3\lib\site-packages\inaSpeechSegmenter\segmenter.py:61: RuntimeWarning: invalid value encountered in subtract
  data = (data - np.mean(data, axis=1).reshape((len(data), 1))) / np.std(data, axis=1).reshape((len(data), 1))
C:\Users\Paul\anaconda3\lib\site-packages\numpy\core\_methods.py:229: RuntimeWarning: invalid value encountered in subtract

Loaded the voice encoder model on cpu in 0.01 seconds.


C:\Users\Paul\anaconda3\lib\site-packages\resemblyzer\audio.py:47: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.], sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  frames = librosa.feature.melspectrogram(


In [ ]:
res3

In [ ]:
tab=pd.DataFrame(res3).to_csv('Equipe 2 - Résultats-2.csv',';')